In [171]:
from src.gru.vae_gru import EncoderDecoder
from src.gru.dataset import PredictionDataset
from src.utils.vectorizer import SELFIESVectorizer
import selfies as sf
from torch.utils.data import DataLoader
import rdkit.Chem as Chem
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
from tqdm import tqdm
vectorizer = SELFIESVectorizer(pad_to_len=128)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#--------------------------------------------------------------------------#

model_path = 'models/fixed_cce_3_layers/epoch_175.pt'
data_path = 'data/GRU_data/5ht1a_fps.parquet'

encoding_size = 512
hidden_size = 512
num_layers = 3
dropout = 0 # dropout must be equal 0 if num_layers = 1
teacher_ratio = 0.5

batch_size = 100

name = 'fixed_cce_3_layers_epoch_175'

#-------------------------------------------------------------------------#

In [172]:
model = EncoderDecoder(
    fp_size=4860,
    encoding_size=encoding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    teacher_ratio = teacher_ratio).to(device)

model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))

df = pd.read_parquet(data_path)

In [173]:
def get_predictions(model, df):
    dataset = PredictionDataset(df, vectorizer)
    loader = DataLoader(dataset, shuffle=False, batch_size=batch_size, drop_last=True)
    preds_smiles = []
    for X in loader:
        X = X.to(device)
        preds = model(X, None, teacher_forcing=False)
        preds = preds.detach().cpu().numpy()
        fps = df.fps.apply(eval).tolist()
        for seq in preds:
            selfie = vectorizer.devectorize(seq, remove_special=True)
            try:
                preds_smiles.append(sf.decoder(selfie))
            except:
                preds_smiles.append('C')
    return preds_smiles

In [174]:
def filter_out_nondruglike(predictions, threshold):
    raw_mols = [Chem.MolFromSmiles(s) for s in predictions]
    raw_qeds = [Chem.QED.qed(m) for m in raw_mols]
    filtered_mols = []
    filtered_qeds = []
    
    for i, value in enumerate(raw_qeds):
        if (value > threshold):
            filtered_mols.append(raw_mols[i])
            filtered_qeds.append(value)
    return filtered_mols, filtered_qeds

predictions = get_predictions(model, df)
predictions_druglike, qeds = filter_out_nondruglike(predictions, 0.4)

In [175]:
if not os.path.isdir(f'imgs/{name}'):
    os.mkdir(f'imgs/{name}')

i = 0    
while i < 1000:
    mol4 = predictions_druglike[i:(i+4)]
    qed4 = qeds[i:(i+4)]
    qed4 = ['{:.2f}'.format(x) for x in qed4]
    img = Chem.Draw.MolsToGridImage(mol4, molsPerRow=2, subImgSize=(400, 400), 
                                    legends=[f'QED: {qed}' for qed in qed4], returnPNG=False)
    img.save(f'imgs/{name}/{i}.png')
    i += 4

[13:46:15] 

****
Pre-condition Violation
no draw context
Violation occurred on line 214 in file /home/conda/feedstock_root/build_artifacts/rdkit_1666697261712/work/Code/GraphMol/MolDraw2D/MolDraw2DCairo.cpp
Failed Expression: dp_cr
****



RuntimeError: Pre-condition Violation
	no draw context
	Violation occurred on line 214 in file Code/GraphMol/MolDraw2D/MolDraw2DCairo.cpp
	Failed Expression: dp_cr
	RDKIT: 2022.09.1
	BOOST: 1_74
